# Preprocessing our Raw images

In [ ]:
"""
Created on Mon Nov 22, 2022

@author: Alemu Sisay
"""

In [2]:
#loading ML dependancies
import matplotlib.pyplot as plt
from matplotlib import pyplot, image, transforms
from scipy import ndimage

import pandas as pd
import numpy as np
from nibabel.testing import data_path
import os
import nibabel as nib
from tqdm import tqdm
from nilearn.image import resample_img
import nibabel as nb
import SimpleITK as sitk


from nipype.interfaces.slicer.filtering.n4itkbiasfieldcorrection import N4ITKBiasFieldCorrection
from nipype import Node, Workflow
from nipype.interfaces.ants import N4BiasFieldCorrection
#Registration
from nipype.interfaces import fsl
from nipype.testing import example_data
#Skull stripping
from nipype.interfaces.fsl import BET
#Normalization
import nipype.interfaces.spm as spm

In [3]:
Raw_AD = []
Raw_CN = []
Raw_MCI = []
for subdir,dirs,files in os.walk('/Users/alex/Documents/Thesis/Jupyter/Test/Raw/AD'):
    for file in files:
        filepath = subdir + os.sep + file
        if not '.DS_Store' in filepath:    
            Raw_AD.append(filepath)
for subdir,dirs,files in os.walk('/Users/alex/Documents/Thesis/Jupyter/Test/Raw/CN'):
    for file in files:
        filepath = subdir + os.sep + file
        if not '.DS_Store' in filepath:    
            Raw_CN.append(filepath)
for subdir,dirs,files in os.walk('/Users/alex/Documents/Thesis/Jupyter/Test/Raw/MCI'):
    for file in files:
        filepath = subdir + os.sep + file
        if not '.DS_Store' in filepath:    
            Raw_MCI.append(filepath)            

In [4]:
#creating a dataframe for ease of use..
data=pd.DataFrame({'Raw_AD':Raw_AD, 'Raw_CN':Raw_CN, 'Raw_MCI':Raw_MCI})
data.head()

,Raw_AD,Raw_CN,Raw_MCI
0,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...
1,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...
2,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...
3,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...
4,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...,/Users/alex/Documents/Thesis/Jupyter/Test/Raw/...


In [5]:
ref_parent = '/Users/alex/Documents/Thesis/AAL3'
ref_path = os.path.join(ref_parent, "MNI152_T1_1mm.nii.gz")


In [6]:
import tempfile

temp_dir1 = tempfile.TemporaryDirectory(dir = '/Users/alex/Documents/Thesis')
temp_dir2 = tempfile.TemporaryDirectory(dir = '/Users/alex/Documents/Thesis')
temp_dir3 = tempfile.TemporaryDirectory(dir = '/Users/alex/Documents/Thesis')


In [7]:
def create_dir(path):
    if not os.path.isdir(path):
        os.makedirs(path)
    return


In [8]:
parent_dir = '/Users/alex/Documents/Thesis/Jupyter/Test'
data_labels = ["AD", "CN","MCI"]
for label in data_labels:
    dst_final_dir1 = os.path.join(parent_dir,'Preprocessed', label)
    create_dir(dst_final_dir1)

In [9]:
path=os.path.join(parent_dir,'Preprocessed')

In [16]:
parent_dir = '/Users/alex/Documents/Thesis/Jupyter/Test'
data_labels = ["AD", "CN","MCI"]
for label in data_labels:
    dst_label_dir1 = os.path.join(temp_dir1.name, label)
    dst_label_dir2 = os.path.join(temp_dir2.name, label)
    dst_label_dir3 = os.path.join(temp_dir3.name, label)
    dst_final_dir = os.path.join(parent_dir,'Preprocessed', label)
    create_dir(dst_label_dir1)
    create_dir(dst_label_dir2)
    create_dir(dst_label_dir3)
    create_dir(dst_final_dir)
    

In [25]:
class preprocessing():
    """
    Reorientation -> Registration -> Skull stripping -> Bias Correction -> Intensity Correction 
    
    """

    def __init__(self,df):
        self.data=df
        self.biased_index_AD=[]
        self.biased_index_CN=[]
        self.biased_index_MCI=[]
        self.biased_index=[]        
    def create_dir(self, path):
        if not os.path.isdir(path):
            os.makedirs(path)
        return
    def reorient2std(self):
        reorient = fsl.Reorient2Std()
        for i in tqdm(range(len(self.data))):
            for label in data_labels:
                if label == 'AD':
                    in_file = self.data.Raw_AD.iloc[i]
                    out_file = temp_dir1.name + '/AD/' + str(i)+'.nii.gz'
                elif label == 'CN':
                    in_file = self.data.Raw_CN.iloc[i]
                    out_file = temp_dir1.name + '/CN/' + str(i)+'.nii.gz'
                else:
                    in_file = self.data.Raw_MCI.iloc[i]
                    out_file = temp_dir1.name + '/MCI/' + str(i)+'.nii.gz'
                reorient.inputs.in_file = in_file              
                reorient.inputs.out_file = out_file
                res = reorient.run()     
    def registration(self):
        """
        Registration to MINI152 atlas image
        Input: in_file -> input file
               reference -> reference file
        Output: out_file -> path/name of registered file 
        """           
        flt = fsl.FLIRT(bins=256, cost_func='corratio')
        flt.inputs.reference = ref_path
        flt.inputs.searchr_x = [0,0]
        flt.inputs.searchr_y = [0,0]
        flt.inputs.searchr_z = [0,0]
        flt.inputs.dof = 12
        flt.inputs.interp = "spline"
        index_reg = []
        for i in tqdm(range(len(self.data))):
            for label in data_labels:
                input_image = temp_dir1.name + '/'+label+'/' + str(i)+'.nii.gz'
                output_image = temp_dir2.name + '/'+label+'/' + str(i)+'.nii.gz'
                flt.inputs.in_file = input_image
                flt.inputs.out_file = output_image
                index_reg.append(output_image)
                res = flt.run()
        temp_dir1.cleanup()
        print('Registration Done! \n Registrated images stored at : ', temp_dir2.name ,'/')           
    def Skull_stripping(self):
        """
        Skull Stripping
        input: in_file -> input file to skull strip
               out_file ->  name of output skull stripped image
               
        Output: out_file -> path/name of skullstripped file (if generated)
        """
        
        btr = fsl.BET()
        btr.inputs.frac = 0.5
        btr.inputs.robust = True
        btr.inputs.vertical_gradient = 0
        for i in tqdm(range(len(self.data))):
            for label in data_labels:
                input_image = temp_dir2.name + '/'+label+'/' + str(i)+'.nii.gz'
                output_image = temp_dir3.name + '/'+label+'/' + str(i)+'.nii.gz'
                btr.inputs.in_file = input_image
                btr.inputs.out_file = output_image
                res = btr.run()
        temp_dir2.cleanup()
        print('Skull Stripping done! Images stored at: ', temp_dir3.name )
    def bias_correction(self):
        """
        Bias field correction: Corrects the bias using ANTs N4BiasFieldCorrection. 

        :param in_file: input file path
        :param out_file: output file path
        :param dimension: image dimension (2, 3 or 4)
        :return: file path to the bias corrected image
        For more optional parameters: https://github.com/nipy/nipype/blob/master/nipype/interfaces/ants/segmentation.py
        """
        #!mkdir bias_correction
        n4 = N4BiasFieldCorrection()
        n4.inputs.dimension = 3
        n4.inputs.shrink_factor = 3
        n4.inputs.n_iterations = [100, 100, 60, 40]
        n4.inputs.convergence_threshold = 1e-4
        n4.inputs.bspline_fitting_distance = 300
        biased_index,biased_index_AD,biased_index_CN,biased_index_MCI,pat1,pat2,pat3 =[],[], [],[],[],[],[]
        for label in data_labels:
            for i in tqdm(range(len(self.data))):
                input_image = temp_dir3.name + '/'+label+'/' + str(i)+'.nii.gz'
                output_image = path + '/'+label+'/' + str(i)+'_'+label +'.nii.gz'
                if label == 'AD':
                    pat1.append(path +'/'+label+'/' +str(i)+'_'+label +'.nii.gz')
                elif label == 'CN':
                    pat2.append(path +'/'+label+'/' +str(i)+'_'+label +'.nii.gz')                   
                else:
                    pat3.append(path +'/'+label+'/' +str(i)+'_'+label +'.nii.gz')
                   
                n4.inputs.input_image = input_image
                n4.inputs.output_image = output_image
                self.biased_index.append(output_image)         
                res = n4.run()
                
        self.biased_index_AD= [pat1[i] for i in range(len(self.data))]
        self.biased_index_CN= [pat2[i] for i in range(len(self.data))]
        self.biased_index_MCI= [pat3[i] for i in range(len(self.data))]                  
        print('Bias corrected images stored at : ',path)
        temp_dir3.cleanup()
        return self.biased_index_AD,self.biased_index_CN, self.biased_index_MCI
    def intensity_normalization(self):
        for label in data_labels:
            if label == 'AD':
                for i in tqdm(self.biased_index_AD):
                    image = sitk.ReadImage(i)
                    resacleFilter = sitk.RescaleIntensityImageFilter()
                    resacleFilter.SetOutputMaximum(255)
                    resacleFilter.SetOutputMinimum(0)
                    image = resacleFilter.Execute(image)
                    sitk.WriteImage(image,i)
            elif label == 'CN':
                for i in tqdm(self.biased_index_CN):
                    image = sitk.ReadImage(i)
                    resacleFilter = sitk.RescaleIntensityImageFilter()
                    resacleFilter.SetOutputMaximum(255)
                    resacleFilter.SetOutputMinimum(0)
                    image = resacleFilter.Execute(image)
                    sitk.WriteImage(image,i)
            else:
                for i in tqdm(self.biased_index_MCI):
                    image = sitk.ReadImage(i)
                    resacleFilter = sitk.RescaleIntensityImageFilter()
                    resacleFilter.SetOutputMaximum(255)
                    resacleFilter.SetOutputMinimum(0)
                    image = resacleFilter.Execute(image)
                    sitk.WriteImage(image,i)                    
        print('Normalization done. Images stored at: ',path)
        

In [26]:
pre=preprocessing(data)

In [14]:
pre.reorient2std()


100%|███████████████████████████████████████████| 13/13 [00:25<00:00,  1.94s/it]


In [21]:
pre.registration()


100%|██████████████████████████████████████████| 13/13 [26:33<00:00, 122.55s/it]

Registration Done! 
 Registrated images stored at :  /Users/alex/Documents/Thesis/tmp8r447d8b /


In [24]:
pre.Skull_stripping()

100%|███████████████████████████████████████████| 13/13 [03:47<00:00, 17.51s/it]

Skull Stripping done! Images stored at:  /Users/alex/Documents/Thesis/tmpagm67ogt


In [27]:
biased_index_AD,biased_index_CN,biased_index_MCI = pre.bias_correction()


100%|███████████████████████████████████████████| 13/13 [16:23<00:00, 75.68s/it]

Bias corrected images stored at :  /Users/alex/Documents/Thesis/Jupyter/Test/Preprocessed


In [38]:
pre.intensity_normalization()

100%|███████████████████████████████████████████| 13/13 [00:05<00:00,  2.18it/s]

Normalization done. Images stored at:  /Users/alex/Documents/Thesis/Jupyter/Test/Preprocessed


ANTs not found
